### 목표 설정
- 네이버 영화 리뷰 데이터셋을 이용해서 긍정/부정을 판별하는 분류기를 만들어보자
- TF-IDF방법을 이용해서 토큰화를 진행해보자
- Konlpy 한국어 형태소 분석기를 이용해보자
- 단어별 긍/부정 정보를 시각화

In [ ]:
# 내 컴퓨터가 구글 컴퓨터 어느곳에 연결되서 위치하는지
# 내 현재 위치 확인
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 소스코드가 있는 경로로 위치를 변경
%cd /content/drive/MyDrive/Colab Notebooks/SW엔지니어

/content/drive/MyDrive/Colab Notebooks/SW엔지니어


In [ ]:
!pwd

/content/drive/MyDrive/Colab Notebooks/SW엔지니어


In [ ]:
#필요한 라이브러리 불러오기
import pandas as pd

In [ ]:
train = pd.read_csv('./data/ratings_train.csv', encoding="utf-8")

In [ ]:
test = pd.read_csv('./data/ratings_test.csv', encoding='utf-8')

In [ ]:
# train df에 대한 정보를 확인해보자
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
# test df에 대한 정보도 확인
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [ ]:
train.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
# 결측치 확인
train[train['document'].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [ ]:
# 결측치 확인
test[test['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [ ]:
# .dropna : 결측치가 있는 행을 삭제해주는 함수
train.dropna(inplace=True)
test.dropna(inplace=True)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [ ]:
train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
train['document']

,document
0,아 더빙.. 진짜 짜증나네요 목소리
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2,너무재밓었다그래서보는것을추천한다
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
...,...
149995,인간이 문제지.. 소는 뭔죄인가..
149996,평점이 너무 낮아서...
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상


In [ ]:
# 평가데이터의 문제 데이터
test['document']

,document
0,굳 ㅋ
1,GDNTOPCLASSINTHECLUB
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네


In [ ]:
# train /test 데이터셋에서 문제데이터/ 정답데이터 지정

# X : 문제
# y : 정답데이터
# train : 훈련
# test : 평가(테스트)

# 문제 데이터
X_train = train['document']
y_train = train['label']

# 정답 데이터
X_test = test['document']
y_test = test['label']

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((149995,), (49997,), (149995,), (49997,))

### 토큰화
**vectorizer**
- vectorizer는 대상을 벡터화 시켜주는 도구
- 벡터화 : 텍스트나 이미지처럼 '비수치적'인 데이터를 숫자 배열(벡터)로 바꾸는 과정
- 기계 학습 분야에서 주로 텍스트 데이터를 분석하기 위해서
- 여러 머신러닝 / 딥러닝 모델을 학습시키기 위해서는 텍스트나 다른 형태의 데이터를 벡터화 / 수치화 할 필요가 있다.


### BOW(Bag of Word) - 단어가방
- 문서를 벡터로 변환하는 가장 기본적인 방법
- 문서내의 모든 단어를 모아서 가방 안에 넣고, 어떤 단어가 들어있는지 리스트 / 딕셔너리 형태로 '순서-숫자'로 나타내는 방법

In [ ]:
# TF-IDF 기법 사용
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
X_train[0:3]

,document
0,아 더빙.. 진짜 짜증나네요 목소리
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2,너무재밓었다그래서보는것을추천한다


In [ ]:
# TF-IDF 벡터라이저 사용

# 객체 생성
tfidf = TfidfVectorizer()

# 단어 사전 구축
tfidf.fit(X_train[0:3])

TfidfVectorizer()

In [ ]:
# 분리된 단어로 만든 단어사전
tfidf.vocabulary_

{'더빙': 2,
 '진짜': 6,
 '짜증나네요': 7,
 '목소리': 3,
 '포스터보고': 9,
 '초딩영화줄': 8,
 '오버연기조차': 5,
 '가볍지': 0,
 '않구나': 4,
 '너무재밓었다그래서보는것을추천한다': 1}

In [ ]:
temp = TfidfVectorizer()

sugar ='설탕국수를 먹었다.'

# 단어 사전 구축
temp.fit([sugar])

temp.vocabulary_

{'설탕국수를': 1, '먹었다': 0}

- TF-IDF Vectorizer는 기본적으로 어절 단위(문장을 이루는 토막)으로 끊어주기 때문에 한국어 특성상 파악이 어렵다.
- 더 작은 단위인 형태소 별로 나눠서 파악해야 한다.

## 한국어 형태소 분석기 사용
- 한국어 특성에 맞춰 개발된 분석기 사용

In [ ]:
# 라이브러리 설치
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 26.5 MB/s eta 0:00:00


In [ ]:
# 형태소 분석기 불러오기
from konlpy.tag import Kkma

In [ ]:
# 꼬꼬마 객체 생성
Kkma = Kkma()

In [ ]:
Kkma.nouns()